In [1]:
import gym
import numpy as np
import copy, random, os, subprocess, cv2
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import layers, models, regularizers

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
MODEL_NAME = 'model08'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
gs://etsuji-car-racing-v2/model04/
gs://etsuji-car-racing-v2/model05/
gs://etsuji-car-racing-v2/model06/
gs://etsuji-car-racing-v2/model07/


In [3]:
class ApplyL1Weight(layers.Layer):
    def __init__(self, l1=0.01, **kwargs):
        self.filter_shape = None
        self.l1 = l1
        super(ApplyL1Weight, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.kernel = self.add_weight(name='weights', shape=[input_shape[3]],
                                      regularizer=regularizers.L1(self.l1))
        self.filter_shape = input_shape

    def get_config(self):
        config = super().get_config()
        config.update({
            'l1': self.l1,
        })
        return config
    
    def call(self, inputs, **kwargs):
        return inputs * self.kernel
    
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(48, 48, 3), name='cnn_input')
        cnn1 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn2')(pool1)        
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)
        weighted_filters = ApplyL1Weight(name='weighted_filters')(pool2)

        cnn_flatten = layers.Flatten(name='flatten')(weighted_filters)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(2048, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(1024, activation='relu', name='dense2')(hidden1)
        hidden3 = layers.Dense(512, activation='relu', name='dense3')(hidden2)
        q_value = layers.Dense(1, name='output')(hidden3)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [4]:
q_value = QValue()
q_value.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                              

2022-08-03 04:20:49.701007: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-03 04:20:49.701058: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-8-20220801-173936): /proc/driver/nvidia/version does not exist
2022-08-03 04:20:49.702848: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def join_frames(o0, o1, o2):
    gray_image0 = cv2.cvtColor(cv2.resize(o0, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image1 = cv2.cvtColor(cv2.resize(o1, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(cv2.resize(o2, (48, 48)), cv2.COLOR_RGB2GRAY)
    
    return np.array(
        [gray_image0.transpose(),
         gray_image1.transpose(),
         gray_image2.transpose()]).transpose()

In [6]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    o2 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1, o2))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1, o2), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1, o2), a, r, join_frames(o1, o2, o_new)))
        o0, o1, o2 = o1, o2, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [7]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [8]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                            

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=361, total rewards=-0.09891696750915568
epsilon=1.0, episode length=201, total rewards=-0.065771812080482
Training the model...
experience length=562
number of examples=1124
best total reward =  [-100, -0.09891696750915568, -0.065771812080482]
loss = [408149.6875, 34.685325622558594, 41.809669494628906, 18.42458724975586, 11.12362289428711, 8.753081321716309, 15.194820404052734, 7.518411159515381, 11.407185554504395, 6.876156330108643]
iteration 2
epsilon=0.992, episode length=94, total rewards=-0.00632911392403665
epsilon=0.992, episode length=153, total rewards=-0.07432950191567156
epsilon=0.992, episode length=68, total rewards=-0.0034482758620619502
epsilon=0.992, episode length=69, total rewards=-0.006993006992998846
epsilon=0.992, episode length=76, total rewards=-0.007692307692297201
epsilon=0.992, episode length=196, total rewards=-0.0921568627450775
Training the model...
experience length=1218
number of examples=2217
best total reward = 

Copying file://car-racing-v2-3-model08.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.9840800000000001, episode length=294, total rewards=-0.08823529411758124
epsilon=0.9840800000000001, episode length=192, total rewards=-0.007395498392272354
Training the model...
experience length=1769
number of examples=2549
best total reward =  [-0.006993006992998846, -0.007692307692297201, -0.007395498392272354]
loss = [0.6142647862434387, 0.5402365326881409, 1.6553845405578613, 0.354451984167099, 0.48177987337112427, 0.4879624545574188, 0.48838910460472107, 0.47183340787887573, 0.4230238199234009, 0.44345706701278687]
iteration 4
epsilon=0.9762392000000002, episode length=145, total rewards=-0.05454545454542381
epsilon=0.9762392000000002, episode length=137, total rewards=-0.05429553264601972
epsilon=0.9762392000000002, episode length=185, total rewards=-0.012639405204429383
epsilon=0.9762392000000002, episode length=141, total rewards=-0.06572438162542515
Training the model...
experience length=2377
number of examples=2606
best total reward =  [-0.006993006992998846, -0.

Copying file://car-racing-v2-6-model08.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.9607920399200003, episode length=139, total rewards=-0.0627177700348247
epsilon=0.9607920399200003, episode length=119, total rewards=-0.047808764940220544
epsilon=0.9607920399200003, episode length=436, total rewards=-0.06363636363623582
Training the model...
experience length=3956
number of examples=2821
best total reward =  [-0.006993006992998846, -0.007692307692297201, -0.007395498392272354]
loss = [0.30184879899024963, 0.2755516469478607, 0.27085748314857483, 0.26630309224128723, 0.2544651925563812, 0.26254957914352417, 0.25063762068748474, 0.24511770904064178, 0.2605138421058655, 0.2512945532798767]
iteration 7
epsilon=0.9531841195208004, episode length=208, total rewards=-0.06666666666661958
epsilon=0.9531841195208004, episode length=92, total rewards=-0.06923076923075233
epsilon=0.9531841195208004, episode length=125, total rewards=-0.06081504702193219
epsilon=0.9531841195208004, episode length=62, total rewards=-0.06947040498441859
epsilon=0.9531841195208004, episode

...

Training the model...
experience length=4995
number of examples=2799
best total reward =  [-0.0035087719297035302, -0.0031746031745331915, 14.814814814813758]
loss = [0.5184290409088135, 0.3391527235507965, 0.2780553698539734, 0.5514646768569946, 0.2641904056072235, 0.2500668168067932, 0.21785615384578705, 0.1937199831008911, 0.21778850257396698, 0.1849837750196457]
iteration 43
epsilon=0.724527376459317, episode length=610, total rewards=-0.031297709923628875
Training the model...
experience length=4995
number of examples=2608
best total reward =  [-0.0035087719297035302, -0.0031746031745331915, 14.814814814813758]
loss = [0.43638405203819275, 0.3144887089729309, 0.23865336179733276, 0.23739075660705566, 0.19749824702739716, 0.17612534761428833, 0.1677696853876114, 0.1628779172897339, 0.1520271897315979, 0.13730217516422272]
iteration 44
epsilon=0.7192821026947238, episode length=735, total rewards=-0.07058823529434588
Training the model...
experience length=4995
number of examples=27

Copying file://car-racing-v2-45-model08.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.7140892816677766, episode length=626, total rewards=-0.004075235109685965
Training the model...
experience length=4995
number of examples=2800
best total reward =  [-0.0035087719297035302, -0.0031746031745331915, 14.814814814813758]
loss = [0.4386675953865051, 0.30856162309646606, 0.2607424259185791, 0.21739815175533295, 0.20892390608787537, 0.18327337503433228, 0.29377374053001404, 0.14934943616390228, 0.15246066451072693, 0.14797236025333405]
iteration 46
epsilon=0.708948388851099, episode length=157, total rewards=-0.051968503936985266
epsilon=0.708948388851099, episode length=999, total rewards=33.82899628252722
Training the model...
experience length=4995
number of examples=3154
best total reward =  [-0.0031746031745331915, 14.814814814813758, 33.82899628252722]
loss = [0.6406534910202026, 0.4101640582084656, 0.3519234359264374, 0.3139306902885437, 0.2786990702152252, 0.2501019835472107, 0.2353590875864029, 0.19916562736034393, 0.18539586663246155, 0.18553222715854645]
i

Copying file://car-racing-v2-48-model08.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2716
best total reward =  [-0.0031746031745331915, 14.814814814813758, 33.82899628252722]
loss = [0.5421455502510071, 0.39488789439201355, 0.3021458089351654, 0.33136940002441406, 0.2553878724575043, 0.23430569469928741, 0.21284975111484528, 0.20815305411815643, 0.18947002291679382, 0.18624867498874664]
iteration 49
epsilon=0.6938321127538326, episode length=352, total rewards=-0.043589743589682345
epsilon=0.6938321127538326, episode length=626, total rewards=-0.013432835820880146
Training the model...
experience length=4995
number of examples=2976
best total reward =  [-0.0031746031745331915, 14.814814814813758, 33.82899628252722]
